In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import pandas as pd
import os
import re
import time

In [2]:
sc_df=pd.DataFrame(index=range(0),columns=["작성자","승인자","게시기한","승인일자","재공지일자","게시대상","분류","게시형태","제목","내용"])

In [3]:
sc_df

,작성자,승인자,게시기한,승인일자,재공지일자,게시대상,분류,게시형태,제목,내용


In [4]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


'''출처: https://data-make.tistory.com/170 [Data Makes Our Future]'''
#파일명으로 폴더이름 만들때 쓴 함수 진우형 다운로드 할때도 이함수 쓰세요

'출처: https://data-make.tistory.com/170 [Data Makes Our Future]'

In [5]:
def crawl_table(data):
    body_num=((1,2),(1,4),(2,2),(2,4),(3,0),(4,2),(4,4),(4,6),(5,0))
    content_list=[]
    for _ in body_num:
        if _[1]!=0:
            body_path=('body > div > div.page > form > table > tbody > tr:nth-child({0}) > td:nth-child({1})'.format(str(_[0]), str(_[1])))
        else:
            body_path=('body > div > div.page > form > table > tbody > tr:nth-child({}) > td'.format(str(_[0])))
        #print(body_path)
        #print(driver.find_element_by_css_selector(body_path).text)
        content_list.append(driver.find_element_by_css_selector(body_path).text) #앞 테이블 내용
    content_list.append(driver.find_element_by_css_selector('body > div > div.page > form > table > tbody > tr:nth-child(6) > td').text) #내용
    data=data.append(pd.Series(content_list,index=data.columns),ignore_index=True)
    return data
    


In [6]:
def imagecrawler():
    if driver.find_elements_by_xpath('/html/body/div/div[2]/form/table/tbody//img'):
        n=1
        newfoldername=driver.find_element_by_css_selector('body > div > div.page > form > table > tbody > tr:nth-child(5) > td').text
        newfoldername=re.sub('/', '_', newfoldername)# / 를 _로 바꿔주는 정규식
        newfoldername=re.sub('\"','',newfoldername) #"를 없애주는 정규식
        newpath=('./img/{}/'.format(newfoldername))#저장할 경로
        newfile=('./img/{}/{}_{}.jpg'.format(newfoldername,newfoldername,n))#이미지 명까지 포함된 경로
        createFolder(newpath)
        for _ in driver.find_elements_by_xpath('/html/body/div/div[2]/form/table/tbody//img'):
            urlretrieve(_.get_attribute('src'),newfile)
            n+=1

In [7]:
def surfer2(data):
    page_num=1
    while True:
        if page_num==1:
            driver.switch_to.frame("_component")
        tab_num=len(driver.find_elements_by_xpath('//*[@id="Search"]/table/tbody/tr'))#테이블 아이탬 몇개있는지 확인
        print(f"페이지 번호 : {page_num}, 탭 수: {tab_num}")
        for i in range(1,tab_num+1):
            if i!=1:#볼드체 제목 클릭
                driver.switch_to.frame("_component")
            path=('//*[@id="Search"]/table/tbody/tr[%d]/td[3]/b/a' %i)
            try: #일반 제목 클릭
                element=driver.find_element_by_xpath(path)
            except:
                path=('//*[@id="Search"]/table/tbody/tr[%d]/td[3]/a' %i)
                element=driver.find_element_by_xpath(path)

            element.click()#여기가 클릭하는 부분 (이밑으로 크롤러 넣으면 됨)
            data=crawl_table(data)#테이블 긁어오는 함수 (이밑으로 사진 저장, 파일 저장 함수 넣으면 됨)
            imagecrawler()#이미지 긁어오는 함수
            print(path)
            time.sleep(0.1)
            driver.back()
        driver.switch_to.frame("_component")
        btn_next=driver.find_element_by_css_selector('#Search > div.paging > div > a.btn.next')
        #장학금 공지는 div[2]/div/a[5], 지난 장학금 공지는 /div[3]/div/a[7]
        if btn_next.get_attribute('href')[-4:]=='prev':#맨마지막 페이지 도달 하면 break
            print("끝도달")
            break
        btn_next.click()
        page_num+=1
        time.sleep(0.1)
    return data

In [8]:
driver = Chrome()

In [9]:
driver.get("https://portal.korea.ac.kr/front/Intro.kpd")

In [10]:
with open('idpw.txt','r') as f:
    data=f.read()

In [11]:
id_,pw_=re.split(r'(\n)',data)[0],re.split(r'(\n)',data)[2]

In [12]:
driver.find_element_by_name('id').clear()
driver.find_element_by_name('id').send_keys(id_)

In [13]:
driver.find_element_by_name('pw').clear()
driver.find_element_by_name('pw').send_keys(pw_)

In [14]:
time.sleep(1)
driver.find_element_by_name('loginsubmit').click()

In [15]:
#포탈 첫화면에서 등록/장학 클릭
time.sleep(2)
driver.find_element_by_xpath('//*[@id="header"]/div[2]/div/div/ul/li[2]/a').click()

In [16]:
sc_df=surfer2(sc_df)

페이지 번호 : 1, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/b/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/b/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/b/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/b/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 2, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 3, 탭 수: 8
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id

In [17]:
driver.back()

In [18]:
driver.find_element_by_xpath('//*[@id="m102"]/a').click() #장학 클릭

In [19]:
driver.find_element_by_xpath('//*[@id="sm110"]/a').click() #지난 장학금 공지 클릭

In [20]:
year=list(range(2010,2022))#2010년 부터 2021년까지 있음

In [21]:
for k in year:
    driver.switch_to.frame("_component")    
    driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/div/input[1]').clear()#년도창비우기
    driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/div/input[1]').send_keys(k)#년도 입력
    driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/span[2]/input').click()#검색 클릭
    driver.find_element_by_css_selector('#Search > div.paging > div > a.btn.first').click()#1페이지로 초기화
    driver.find_element_by_xpath('//*[@id="Search"]/table/tbody/tr[1]/td[3]/a').click()#아무페이지나 먼저 클릭
    driver.back()
    sc_df=surfer2(sc_df)
    driver.back()
    sc_df
    #break
    

페이지 번호 : 1, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 2, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 3, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Searc

//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 11, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 12, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr

//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 2, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 3, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2

//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 3, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 4, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[1

//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 4, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 5, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9

...

//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 13, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 14, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr

//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 7, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 8, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9

//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 25, 탭 수: 3
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
끝도달
페이지 번호 : 1, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 2, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="S

페이지 번호 : 18, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 19, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 20, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Se

//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 10, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 11, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr

In [ ]:
driver.back()

In [22]:
sc_df

,작성자,승인자,게시기한,승인일자,재공지일자,게시대상,분류,게시형태,제목,내용
0,최지연 (학생지원부),최지연 (학생지원부),2022-02-28,2021-01-26 (화) 오후 04:40,2021-03-23,"서울(학부, 직원),의료원(학부, 직원)",장학금 공지사항,일반,[교내-장학일정] 2021학년도 장학금 일정 안내(학부),2021학년도 장학금 일정 안내(학부)\n - 본 일정표는 확정된 일정이 아닌...
1,곽태완 (학생지원부),곽태완 (학생지원부),2021-12-31,2021-01-14 (목) 오후 04:04,2021-03-23,"서울(학부, 직원),의료원(학부, 직원)",장학금 공지사항,일반,[선발결과] 2021학년도 교외장학재단 신규장학생 선발 결과 안내,[선발결과] 2021학년도 교외장학재단 신규장학생 선발 결과 안내\n\n\n※ 장학...
2,김정현 (학생지원부),김정현 (학생지원부),2021-08-31,2021-02-16 (화) 오전 11:56,2021-03-23,"서울(학부, 교수, 직원)",장학금 공지사항,일반,[선발결과] 2021학년도 기금장학금 신규장학생 선발 안내,portal.korea.ac.kr - 로그인 - 장학금공지사항 안내 - 상세내용 참...
3,김정현 (학생지원부),김정현 (학생지원부),2022-02-28,2021-02-21 (일) 오후 07:57,2021-03-22,"서울(강사, 학부, 교수, 직원)",장학금 공지사항,일반,[선발결과] 2021년 국가근로장학생/특별근로장학생 선발 및 사전교육 안내,2021년 국가근로장학금 사전교육자료 및 위치기반 모바일 출근부 앱 교육자료(학생...
4,최지연 (학생지원부),최지연 (학생지원부),2021-07-23,2021-07-09 (금) 오후 04:58,2021-07-09,"서울(학부, 직원),의료원(학부, 직원)",장학금 공지사항,일반,[국가-7/23] 2021년 2학기 푸른등대 기부장학금 신규장학생 선발 안내,2021학년도 2학기 푸른등대 기부장학금 신규장학생 선발 안내\n 1. 2021학...
...,...,...,...,...,...,...,...,...,...,...
2191,곽태완 (학생지원부),곽태완 (학생지원부),2021-02-04,2021-01-19 (화) 오전 10:25,2021-01-19,"서울(학부, 직원),의료원(학부, 직원)",지난 장학금 공지사항,일반,[교외-2/4] (남자 기숙사) 21-1 숭덕육영회 기숙사 장학생 선발 안내(재단지원),2021학년도 1학기 숭덕육영회 기숙사 장학생 선발 안내\n\n1. 인원 : 전체 ...
2192,곽태완 (학생지원부),곽태완 (학생지원부),2021-01-22,2021-01-14 (목) 오후 03:35,2021-01-14,"서울(학부, 직원),의료원(학부, 직원)",지난 장학금 공지사항,일반,[교외-1/22] 2021학년도 정수장학회 장학생 선발 안내(학교지원),2021학년도 정수장학회 장학생 선발 안내\n\n\n1. 선발인원 : 총 2명(인문...
2193,글로벌서비스센터 (글로벌서비스센터),강영선 (글로벌서비스센터),2021-01-15,2021-01-06 (수) 오후 04:13,2021-01-14,"서울(강사, 학부, 교수, 직원)",지난 장학금 공지사항,일반,[GSC] Spring 2021 Scholarships for Int'l Students,[글로벌서비스센터] 2021학년도 1학기 외국인 학부생 장학금 신청안내\n\n글로벌...
2194,최지연 (학생지원부),최지연 (학생지원부),2021-01-22,2021-01-08 (금) 오후 04:52,2021-01-08,"서울(학부, 직원),의료원(학부, 직원)",지난 장학금 공지사항,일반,[기숙사-1/22] 한국장학재단-은행권 대학생 연합기숙사 2021년 1학기 정기모집 안내,자세한 사항은 첨부된 모집 안내문을 참고하여 주시기 바랍니다.


In [32]:
sc_df['제목']

0                         [교내-장학일정] 2021학년도 장학금 일정 안내(학부)
1                    [선발결과] 2021학년도 교외장학재단 신규장학생 선발 결과 안내
2                        [선발결과] 2021학년도 기금장학금 신규장학생 선발 안내
3               [선발결과] 2021년 국가근로장학생/특별근로장학생 선발 및 사전교육 안내
4              [국가-7/23] 2021년 2학기 푸른등대 기부장학금 신규장학생 선발 안내
                              ...                        
2191     [교외-2/4] (남자 기숙사) 21-1 숭덕육영회 기숙사 장학생 선발 안내(재단지원)
2192              [교외-1/22] 2021학년도 정수장학회 장학생 선발 안내(학교지원)
2193    [GSC] Spring 2021 Scholarships for Int'l Students
2194    [기숙사-1/22] 한국장학재단-은행권 대학생 연합기숙사 2021년 1학기 정기모집 안내
2195               [교외-1/14] 2021학년도 우양재단 장학생 선발 안내(재단지원)
Name: 제목, Length: 2196, dtype: object

In [46]:
for i in range(len(sc_df)):
    temp=sc_df['제목'][i]
    temp=re.sub(r'/','_',temp)
    temp=re.sub('\"','',temp)
    sc_df.iloc[i]['제목']=temp

In [47]:
sc_df['제목']

0                         [교내-장학일정] 2021학년도 장학금 일정 안내(학부)
1                    [선발결과] 2021학년도 교외장학재단 신규장학생 선발 결과 안내
2                        [선발결과] 2021학년도 기금장학금 신규장학생 선발 안내
3               [선발결과] 2021년 국가근로장학생_특별근로장학생 선발 및 사전교육 안내
4              [국가-7_23] 2021년 2학기 푸른등대 기부장학금 신규장학생 선발 안내
                              ...                        
2191     [교외-2_4] (남자 기숙사) 21-1 숭덕육영회 기숙사 장학생 선발 안내(재단지원)
2192              [교외-1_22] 2021학년도 정수장학회 장학생 선발 안내(학교지원)
2193    [GSC] Spring 2021 Scholarships for Int'l Students
2194    [기숙사-1_22] 한국장학재단-은행권 대학생 연합기숙사 2021년 1학기 정기모집 안내
2195               [교외-1_14] 2021학년도 우양재단 장학생 선발 안내(재단지원)
Name: 제목, Length: 2196, dtype: object

In [48]:
sc_df.to_excel("장학금데이터.xlsx")